<a href="https://colab.research.google.com/github/merucode/Stock/blob/11_stock_info/11_theme_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependency

In [ ]:
!pip install pykrx

# Plot

In [ ]:
from pykrx import stock           # 데이터 : 주식 데이터 전송
import datetime                   # 그래프 : 문자열 시간 전환
import matplotlib.pyplot as plt   # 그래프
%matplotlib inline

tickers_KOSDAQ = stock.get_market_ticker_list(market="KOSDAQ")
tickers_KOSPI = stock.get_market_ticker_list(market="KOSPI")
tickers = tickers_KOSDAQ + tickers_KOSPI
ticker_dict = dict()

for ticker in tickers:
    stock_name = stock.get_market_ticker_name(ticker)
    ticker_dict[stock_name] = ticker

def convert_ohlcv_format(df):
    df = df.reset_index()
    df = df[['날짜', '시가', '고가', '저가', '종가', '거래량']]
    df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
    return df

def plot_with_issue_date(start_date, end_date, stock_name_lst, issue_date_lst, ticker_dict, adjust=False):
    ### Get ohlcv
    ohlcv_lst = []
    for stock_name in stock_name_lst:
        ohlcv = stock.get_market_ohlcv(start_date, end_date, ticker_dict[stock_name])
        ohlcv = convert_ohlcv_format(ohlcv)
        ohlcv_lst.append(ohlcv)
    ohlcv_kosdaq = stock.get_index_ohlcv(start_date, end_date, "2001")
    ohlcv_kosdaq = convert_ohlcv_format(ohlcv_kosdaq)
    ohlcv_kosdaq['close'] = ohlcv_kosdaq['close'] if adjust==True else 1

    ### Convert str date to datetime date
    issue_date_dt_lst = []
    for issue_date_str in issue_date_lst:
        issue_date_dt_lst.append(datetime.datetime.strptime(issue_date_str, '%Y-%m-%d'))

    ### Plot
    fig, ax = plt.subplots(len(stock_name_lst)+1, 1)
    fig.subplots_adjust(hspace=1)
    plot_color_lst = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']

    for ohlcv, stock_name in zip(ohlcv_lst, stock_name_lst):     # close price line plot
        ax[0].plot(ohlcv['date'], ohlcv['close']/ohlcv['close'][0]/ohlcv_kosdaq['close'], label=stock_name)

    for i, ohlcv in enumerate(ohlcv_lst):     # close price adjust by kosdaq line plot
        ax[i+1].plot(ohlcv['date'], ohlcv['close']/ohlcv_kosdaq['close'], color=plot_color_lst[i])
        ax[i+1].set_title(f'{stock_name_lst[i]}')

    for pt in issue_date_dt_lst:    # add issue date line
        for i in range(0, len(stock_name_lst)+1):
            ax[i].axvline(x=pt, color='#d62728')

    ax[0].legend()
    plt.show()

In [ ]:
start_date = '2017-01-01'
end_date = '2023-10-18'
stock_name_lst = ['크린앤사이언스', '하츠']
issue_date_lst = ['2018-01-01', '2019-01-01', '2020-01-01', '2021-01-01', '2022-01-01', '2023-01-01']

plot_with_issue_date(start_date, end_date, stock_name_lst, issue_date_lst, ticker_dict)